In [1]:
from vUtil.vTime import getNow
from vMysql import MysqlProxy

In [2]:
from getData.getPaperData import getActions, GetData

In [15]:
db('''
create temporary table tmp
select golaxy_paper_id as id from papers limit 10;
''')

""


In [16]:
db.select('*', 'tmp')

tmp,id
0,1032528390384844803
1,1032528390384844804
2,1032528390384844806
3,1032528390384844807
4,1032528390384844808
5,1032528390384844809
6,1032528390384844812
7,1032528390384844815
8,1032528390384844816
9,1032528390384844818


In [10]:
from incEs.esPaper import this, thisEn, table, idCol

In [11]:
getData = GetData(this, thisEn, table, idCol)

In [17]:
d, p = getData(db, 'tmp', 0, 5)

In [18]:
d

,gid,title,keywords,paper_year,paper_date,abstract
0,1032528390384844803,大电流密度碳纳米管阴极的生长及场发射性能研究,碳纳米管;场发射;CVD,2004,None,"研究了碳纳米管作为大电流密度场发射阴极的CVD生长情况与场发射性能.结果表明,通过CVD生长..."
1,1032528390384844804,基于倾斜光学元件的共形光学窗口瞄视误差校正,光学设计;共形光学;光学窗口;瞄视误差;倾斜,2016,None,"为校正共形光学窗口引入的随扫描视角变化而变化的动态瞄视误差,提出基于倾斜光学元件的瞄视误差校..."
2,1032528390384844806,非均匀柱对称介质中的余弦光束,非均匀介质;柱对称介质;余弦光束;折射率分布,2001,None,"在一般情况下证明了余弦光束是柱对称介质中SVA近似下波动方程的本征解,并给出了折射率分布函数..."
3,1032528390384844807,基于DSP/FPGA的模块化柔性关节轻型机械臂,模块化;柔性关节;轻型机械臂;谐波齿轮;DSPFPGA,2010,None,介绍了模块化柔性关节轻型机械臂的机械系统设计、传感器系统设计以及电气系统设计.整个机械结构设...
4,1032528390384844808,高校后勤管理社会化改革模式比较研究,高校;后勤社会化;改革模式,2014,None,"传统高校后勤管理已经不适应当今高等教育快速发展的要求,社会化改革势在必行.通过对多个改革模式..."


In [19]:
p

,gid,scholar_id,names
0,1032528390384844803,1032637347824795649;1115639005386768487;103263...,"[娄朝刚, 雷威, 朱春晖, 张晓兵]"
1,1032528390384844804,1032528364128504121;1032528364128503507,"[崔庆丰, 赵春竹]"
2,1032528390384844806,1032528364128502381;1032637348688822276,"[吕百达, 王喜庆]"
3,1032528390384844807,1114011708330213486;1032528364128509163;107820...,"[刘宏, 谢宗武, 熊根良]"
4,1032528390384844808,1032528364128506650,[董磊]


In [20]:
getActions(d,p,'landinn_paper')

[{'_op_type': 'index',
  '_index': 'landinn_paper',
  '_id': 1032528390384844803,
  '_source': {'id': 1032528390384844803,
   'title': '大电流密度碳纳米管阴极的生长及场发射性能研究',
   'abstract': '研究了碳纳米管作为大电流密度场发射阴极的CVD生长情况与场发射性能.结果表明,通过CVD生长的碳纳米管的直径与催化剂颗粒的直径相近,其生长方向是随机的.根据薄膜厚度与催化剂颗粒的关系,认为通过控制催化剂薄膜的厚度可能会达到调节碳纳米管直径的目的.在实验中获得的碳纳米管具备了良好的场发射性能,在直径为0.13 mm的圆形面积上获得的碳纳米管场发射平均电流密度达到1.28 A/cm2.',
   'keywords': '碳纳米管;场发射;CVD',
   'scholars': '1032637347824795649;1115639005386768487;1032637348290363394;1102765153900167171',
   'year': '2004',
   'names': [{'name': '娄朝刚'},
    {'name': '雷威'},
    {'name': '朱春晖'},
    {'name': '张晓兵'}]}},
 {'_op_type': 'index',
  '_index': 'landinn_paper',
  '_id': 1032528390384844804,
  '_source': {'id': 1032528390384844804,
   'title': '基于倾斜光学元件的共形光学窗口瞄视误差校正',
   'abstract': '为校正共形光学窗口引入的随扫描视角变化而变化的动态瞄视误差,提出基于倾斜光学元件的瞄视误差校正.建立共形光学窗口的瞄视误差模型,并分析计算共形光学窗口引入的动态瞄视误差.阐明利用倾斜光学元件校正共形光学窗口瞄视误差的原理,并通过在共形光学窗口后加入倾斜的球面透镜予以证明.给出一个设计实例,通过将校正共形光学窗口像差的校正元件倾斜,实现共形光学窗口的瞄视误差校正,且并未影响共形光学窗口像差的校正.',
   'key

In [14]:
db.close()

In [2]:
from tmpIncTable.tmpPatents import TmpPatents
from tmpIncTable.tmpProjects import TmpProjects
from tmpIncTable.tmpPapers import TmpPapers
from tmpIncTable.tmpEnterprises import TmpEnterprises
from tmpIncTable.tmpScholars import TmpScholars
from tmpIncTable.tmpScholarsAbroad import TmpScholarsAbroad
from tmpIncTable.tmpProducts import TmpProducts
from tmpIncTable.tmpSoftwareCopyrights import TmpSoftwareCopyrights

In [9]:
import sys
sys.path += ['..']

In [3]:
from timeManager import TimeManagerMysql

In [8]:
tm = TimeManagerMysql('sync_record', 'sync_time', 'es_flag')
tm

< TimeManagerMysql {'last': Timestamp('2021-06-07 16:00:00'), 'now': Timestamp('2021-06-10 16:00:00')} >

In [7]:
print(tm.db.history)


            update `sync_record`
            set `es_flag` = 1
            where not `es_flag`
            and `sync_time` <= Timestamp('2021-06-10 16:00:00')
        


In [6]:
tm.close()

In [15]:
flagCol = 'es_flag'
timeCol = 'sync_time'
table = 'sync_record'
a = tm.db.select(timeCol, table, 1, where=flagCol, order=f'by {timeCol} desc')

In [22]:
a.

('sync_time',
 0   2021-06-07 16:00:00
 Name: sync_time, dtype: datetime64[ns])

In [4]:
db = MysqlProxy()

In [11]:
order\ by = 1

SyntaxError: unexpected character after line continuation character (<ipython-input-11-16c82fe4237d>, line 1)

In [9]:
db.select('*', 'landinn_restore.sync_record', 1, where='es_flag', order ='by id desc')['sync_time'].item()

Timestamp('2021-06-07 16:00:00')

In [7]:
db.select('*', '', 1, where='es_flag order by id desc')['sync_time'].item()
db.sql('''
select * from landinn_restore.sync_record where es_flag order by id desc 
''')

Timestamp('2021-06-07 16:00:00')

In [8]:
db.select('*', 'landinn_restore.sync_record', 1, where='not es_flag order by id desc')['sync_time'].item()

Timestamp('2021-06-10 14:00:00')

In [9]:
tLast = '2021-06-01'
tNow = getNow()

In [10]:
tmp = TmpPatents(db)

In [11]:
tmp.start(tLast, tNow)

In [12]:
len(tmp)

31131

In [13]:
tmp.close()

In [13]:
from tmpIncTable.tmpTemplet import getSqlInsert
from utils import stdSqlData

In [5]:
tableTmp = 'temp_db.tmp_patents'
print(getSqlInsert(tableTmp, 'patent', 'golaxy_patent_id', containDel=0),)


    replace into temp_db.tmp_patents
    (
    	select  golaxy_patent_id as id from patent
        where %s <= if (updated_at is null, created_at, updated_at)
            and if (updated_at is null, created_at, updated_at) < %s
            and !ifnull(is_deleted, 0)
    );
    


In [6]:
sCreate = '''
create table if not exists %s
(id bigint not null, primary key (id));
'''
sRmTmp = '''
drop table if exists %s;
'''

In [7]:
db.sql(sRmTmp % tableTmp, 1)

""


In [8]:
db.sql(sCreate % tableTmp, 1)

""


In [8]:
tLast = '2021-06-03'
tNow = getNow()

In [15]:
db.sql(getSqlInsert(tableTmp, 'patent', 'golaxy_patent_id', containDel=0) % (tLast, tNow))

""


In [20]:
db.close()

In [9]:
tmp = TmpPatents(db)

In [10]:
tmp.start(tLast, tNow)

In [11]:
len(tmp)

1016

In [12]:
tmp.close()

In [20]:
where = '1'
where = [
        'ifnull(is_deleted,0)',
        (f'{stdSqlData(tLast)} <= if (updated_at is null, created_at, updated_at)' if tLast else ''),
        (f'if (updated_at is null, created_at, updated_at) < {stdSqlData(tNow)}' if tNow else ''),
        (f'({where})' if where else ''),
    ]
where = '\n\tand '.join([x for x in where if x])
print(where)

ifnull(is_deleted,0)
	and '2021-06-03' <= if (updated_at is null, created_at, updated_at)
	and if (updated_at is null, created_at, updated_at) < '2021-06-03 17:32:02'
	and (1)
